In [28]:
import cv2
import numpy as np
from itertools import combinations

class SfMCalibrator:
    def __init__(self, max_iter=100, reproj_thresh=2.0, pyr_levels=2):
        self.max_iter = max_iter
        self.reproj_thresh = reproj_thresh
        self.images = []
        self.K = None
        self.dist_coeffs = np.zeros(5, dtype=np.float64)  # ✅ 수정됨
        self.rotations = []
        self.translations = []
        self.image_size = None
        self.pyr_levels = pyr_levels

    def __call__(self, image_list):
        self.set_images(image_list)
        return self.run_pipeline()

    def set_images(self, image_list):
        self.images = [img.copy() for img in image_list if img is not None]
        if not self.images:
            raise ValueError("No valid images")
        self._init_camera_params()

    def _init_camera_params(self):
        h, w = self.images[0].shape[:2]
        self.image_size = (w, h)
        gray = cv2.cvtColor(self.images[0], cv2.COLOR_BGR2GRAY)
        initial_f = self.image_size[0]
        self.K = np.array([[initial_f, 0, w/2],
                           [0, initial_f, h/2],
                           [0, 0, 1]], dtype=np.float64)
        self.rotations = [np.eye(3, dtype=np.float64)]
        self.translations = [np.zeros((3, 1), dtype=np.float64)]

    def _extract_features(self):
        sift = cv2.SIFT_create()
        features = []
        for img in self.images:
            pyr = [img]
            for _ in range(self.pyr_levels):
                pyr.append(cv2.pyrDown(pyr[-1]))
            kp, des = [], []
            for p_img in pyr:
                gray = cv2.cvtColor(p_img, cv2.COLOR_BGR2GRAY)
                k, d = sift.detectAndCompute(gray, None)
                if k:
                    scale = img.shape[1] / p_img.shape[1]
                    k = [cv2.KeyPoint(x.pt[0]*scale, x.pt[1]*scale, x.size*scale, 
                                     x.angle, x.response, x.octave, x.class_id) for x in k]
                    kp.extend(k)
                    des.append(d) if d is not None else None
            des = np.vstack(des) if des else None
            features.append((kp, des))
        return features

    def _match_features(self, features):
        matcher = cv2.BFMatcher()
        matches = []
        n = len(features)
        for i in range(n-1):
            kp1, des1 = features[i]
            kp2, des2 = features[i+1]
            if des1 is None or des2 is None:
                continue

            # 변경된 부분: OpenCV의 knnMatch 사용
            raw_matches = matcher.knnMatch(des1, des2, k=2)
            good = []
            for m, n in raw_matches:
                if m.distance < 0.65 * n.distance:
                    good.append(m)

            matches.append((i, i+1, kp1, kp2, good))
        return matches


    def _undistort_points(self, pts):
        pts = np.asarray(pts, dtype=np.float32).reshape(-1, 1, 2)
        return cv2.undistortPoints(pts, self.K, self.dist_coeffs).reshape(-1, 2)  # ✅ P=None (default)


    def _estimate_pose(self, kp1, kp2, matches):
        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
        pts1u = self._undistort_points(pts1)
        pts2u = self._undistort_points(pts2)
        E, mask = cv2.findEssentialMat(pts1u, pts2u, self.K, cv2.RANSAC, 0.999, self.reproj_thresh)
        _, R, t, _ = cv2.recoverPose(E, pts1u, pts2u, self.K, mask=mask)
        return R, t, mask

    def _triangulate(self, R, t, kp1, kp2, matches, mask):
        P1 = self.K @ np.hstack([np.eye(3), np.zeros((3,1))])
        P2 = self.K @ np.hstack([R, t])
        mask = mask.ravel().astype(bool)
        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])[mask]
        pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])[mask]
        pts4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
        return (pts4d[:3]/pts4d[3]).T

    def _lm_optimize(self, params, points_3d, points_2d, camera_indices):
        x = params.astype(np.float64)
        lambda_ = 1e-2
        for _ in range(self.max_iter):
            residuals = self._compute_residuals(x, points_3d, points_2d, camera_indices)
            J = self._compute_jacobian(x, points_3d, points_2d, camera_indices)
            grad = J.T @ residuals
            hess = J.T @ J
            delta = np.linalg.solve(hess + lambda_*np.eye(len(x)), -grad)
            x_new = x + delta
            new_residuals = self._compute_residuals(x_new, points_3d, points_2d, camera_indices)
            if np.linalg.norm(new_residuals) < np.linalg.norm(residuals):
                x = x_new
                lambda_ *= 0.1
            else:
                lambda_ *= 10
        return x

    def _compute_residuals(self, params, points_3d, points_2d, camera_indices):
        f, k1, k2 = params
        cx, cy = self.image_size[0] / 2, self.image_size[1] / 2
        K = np.array([[f, 0, cx],
                    [0, f, cy],
                    [0, 0, 1]])

        residuals = []
        for i, (pt3d, pt2d) in enumerate(zip(points_3d, points_2d)):
            R = self.rotations[camera_indices[i]]
            t = self.translations[camera_indices[i]]
            proj = K @ (R @ pt3d.reshape(3, 1) + t)
            z = proj[2, 0]

            if z <= 1e-6:
                residuals.extend([0.0, 0.0])
                continue

            proj = proj[:2, 0] / z
            x = proj[0] - cx
            y = proj[1] - cy
            r2 = x ** 2 + y ** 2
            proj = proj * (1 + k1 * r2 + k2 * r2 ** 2) + np.array([cx, cy])
            residuals.extend(proj - pt2d)

        return np.array(residuals)



    def _compute_jacobian(self, params, points_3d, points_2d, camera_indices, eps=1e-6):
        n_params = len(params)
        base = self._compute_residuals(params, points_3d, points_2d, camera_indices)
        J = np.zeros((len(base), n_params))

        for i in range(n_params):
            perturbed = params.copy()
            perturbed[i] += eps
            delta = self._compute_residuals(perturbed, points_3d, points_2d, camera_indices)

            if delta.shape == base.shape:
                J[:, i] = (delta - base) / eps
            else:
                raise ValueError(f"Jacobian shape mismatch: {delta.shape} vs {base.shape}")

        return J



    def run_pipeline(self):
        features = self._extract_features()
        matches = self._match_features(features)
        points_3d, points_2d, indices = [], [], []

        for i, j, kp1, kp2, ms in matches:
            if len(ms) < 4:
                continue
            R, t, mask = self._estimate_pose(kp1, kp2, ms)
            pts3d = self._triangulate(R, t, kp1, kp2, ms, mask)
            if len(pts3d) == 0:
                continue
            inliers = np.where(mask.ravel() == 1)[0]
            points_3d.append(pts3d)
            points_2d.extend([kp1[ms[i].queryIdx].pt for i in inliers])
            indices.extend([i] * len(inliers))
            if len(self.rotations) <= j:
                self.rotations.append(R @ self.rotations[i])
                self.translations.append(self.translations[i] + t)

        all_points_3d = np.vstack(points_3d)
        all_points_2d = np.array(points_2d)
        camera_indices = np.array(indices)

        # 깊이 마스킹
        valid_mask = all_points_3d[:, 2] > 1e-6
        all_points_3d = all_points_3d[valid_mask]
        all_points_2d = all_points_2d[valid_mask]
        camera_indices = camera_indices[valid_mask]

        # ✅ cx, cy 고정하고 f, k1, k2만 추정
        params = np.array([self.K[0, 0], 0, 0])  # f, k1, k2
        params = self._lm_optimize(params, all_points_3d, all_points_2d, camera_indices)

        # ✅ 최종 intrinsic matrix 구성
        cx, cy = self.image_size[0] / 2, self.image_size[1] / 2
        self.K = np.array([[params[0], 0, cx],
                        [0, params[0], cy],
                        [0, 0, 1]])
        self.dist_coeffs = np.array(params[1:3])
        return self.K, self.dist_coeffs, self.rotations




In [29]:
print(img1.shape)

(1329, 2000, 3)


In [31]:
# img1 = cv2.imread('./image/problem_2/3.jpg')
# img2 = cv2.imread('./image/problem_2/4.jpg')
# img3 = cv2.imread('./image/problem_2/5.jpg')
# img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
# img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
# img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
# image_list = [img1, img2, img3]
# calibration = SfMCalibrator()
# K, dist_coeffs, rotations = calibration(image_list)
# print("Intrinsic Matrix (K):", K)
# print("Distortion Coefficients:", dist_coeffs)
# print("Rotations:", rotations)

In [ ]:
img1 = cv2.imread('./image/problem_3/1.jpg')
img2 = cv2.imread('./image/problem_3/2.jpg')
img3 = cv2.imread('./image/problem_3/3.jpg')
img4 = cv2.imread('./image/problem_3/4.jpg')
img5 = cv2.imread('./image/problem_3/5.jpg')
img6 = cv2.imread('./image/problem_3/6.jpg')
img7 = cv2.imread('./image/problem_3/7.jpg')
img8 = cv2.imread('./image/problem_3/8.jpg')
img9 = cv2.imread('./image/problem_3/9.jpg')
img10 = cv2.imread('./image/problem_3/10.jpg')
img11 = cv2.imread('./image/problem_3/11.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img5 = cv2.cvtColor(img5, cv2.COLOR_BGR2RGB)
img6 = cv2.cvtColor(img6, cv2.COLOR_BGR2RGB)
img7 = cv2.cvtColor(img7, cv2.COLOR_BGR2RGB)
img8 = cv2.cvtColor(img8, cv2.COLOR_BGR2RGB)
img9 = cv2.cvtColor(img9, cv2.COLOR_BGR2RGB)
img10 = cv2.cvtColor(img10, cv2.COLOR_BGR2RGB)
img11 = cv2.cvtColor(img11, cv2.COLOR_BGR2RGB)
image_list = [img1, img2, img3, img4, img5, img6, img7, img8, img9, img10, img11]
calibration = SfMCalibrator()
K, dist_coeffs, rotations = calibration(image_list)
print("Intrinsic Matrix (K):", K)
print("Distortion Coefficients:", dist_coeffs)
print("Rotations:", rotations)
